In [1]:
# ignore warning
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# visualize 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# working with dates
from datetime import datetime

# to evaluated performance using rmse
from sklearn.metrics import mean_squared_error
from math import sqrt 

# for tsa 
import statsmodels.api as sm

# holt's linear trend model. 
from statsmodels.tsa.api import Holt

In [2]:
all_items = pd.read_csv('all_items.csv')

In [3]:
apparel = pd.read_csv('apparel.csv')

In [4]:
energy = pd.read_csv('energy.csv')

In [5]:
food = pd.read_csv('food.csv')

In [7]:
gas = pd.read_csv('gasoline.csv')

In [8]:
medical = pd.read_csv('medical.csv')

In [10]:
transportation = pd.read_csv('transportation.csv')

In [11]:
all_items.head()

,Series ID,Year,Period,Label,Value,1-Month % Change
0,CUSR0000SA0,1957,M01,1957 Jan,27.67,0.1
1,CUSR0000SA0,1957,M02,1957 Feb,27.80,0.5
2,CUSR0000SA0,1957,M03,1957 Mar,27.86,0.2
3,CUSR0000SA0,1957,M04,1957 Apr,27.93,0.3
4,CUSR0000SA0,1957,M05,1957 May,28.00,0.3


In [12]:
apparel.head()

,Series ID,Year,Period,Label,Value,1-Month % Change
0,CUSR0000SAA,1957,M01,1957 Jan,44.3,0.0
1,CUSR0000SAA,1957,M02,1957 Feb,44.3,0.0
2,CUSR0000SAA,1957,M03,1957 Mar,44.5,0.5
3,CUSR0000SAA,1957,M04,1957 Apr,44.4,-0.2
4,CUSR0000SAA,1957,M05,1957 May,44.5,0.2


In [13]:
energy.head()

,Series ID,Year,Period,Label,Value,1-Month % Change
0,CUSR0000SA0E,1957,M01,1957 Jan,21.3,N/A
1,CUSR0000SA0E,1957,M02,1957 Feb,21.4,0.5
2,CUSR0000SA0E,1957,M03,1957 Mar,21.5,0.5
3,CUSR0000SA0E,1957,M04,1957 Apr,21.6,0.5
4,CUSR0000SA0E,1957,M05,1957 May,21.6,0.0


In [14]:
food.head()

,Series ID,Year,Period,Label,Value,1-Month % Change
0,CUSR0000SAF1,1957,M01,1957 Jan,28.4,-0.4
1,CUSR0000SAF1,1957,M02,1957 Feb,28.7,1.1
2,CUSR0000SAF1,1957,M03,1957 Mar,28.6,-0.3
3,CUSR0000SAF1,1957,M04,1957 Apr,28.6,0.0
4,CUSR0000SAF1,1957,M05,1957 May,28.7,0.3


In [15]:
gas.head()

,Series ID,Year,Period,Label,Value,1-Month Net Change,1-Month % Change
0,CUSR0000SETB01,1967,M01,1967 Jan,26.3,N/A,N/A
1,CUSR0000SETB01,1967,M02,1967 Feb,26.2,NaN,-0.4
2,CUSR0000SETB01,1967,M03,1967 Mar,26.1,NaN,-0.4
3,CUSR0000SETB01,1967,M04,1967 Apr,26.4,NaN,1.1
4,CUSR0000SETB01,1967,M05,1967 May,26.4,NaN,0.0


In [16]:
medical.head()

,Series ID,Year,Period,Label,Value,1-Month % Change
0,CUSR0000SAM2,1957,M01,1957 Jan,16.7,0.6
1,CUSR0000SAM2,1957,M02,1957 Feb,16.7,0.0
2,CUSR0000SAM2,1957,M03,1957 Mar,16.8,0.6
3,CUSR0000SAM2,1957,M04,1957 Apr,16.9,0.6
4,CUSR0000SAM2,1957,M05,1957 May,16.9,0.0


In [17]:
transportation.head()

,Series ID,Year,Period,Label,Value,1-Month % Change
0,CUSR0000SAS4,1957,M01,1957 Jan,23.5,0.9
1,CUSR0000SAS4,1957,M02,1957 Feb,23.7,0.9
2,CUSR0000SAS4,1957,M03,1957 Mar,23.7,0.0
3,CUSR0000SAS4,1957,M04,1957 Apr,23.8,0.4
4,CUSR0000SAS4,1957,M05,1957 May,23.9,0.4
